In [ ]:
import torch
import numpy as np
import pickle

from vocab import Vocab, load_vocab
from common import Data, Split, Batches, load_data, encode_y, load_split
from utils import ProgressBar, evaluate
from pack import Pack
from model import Model, Model2, TextOnlyModel
from train import train_epoches

%load_ext autoreload
%autoreload 2

In [ ]:
genre_list = pickle.load(open("../data/tmdb_genres_list.pkl", 'rb'))

GENRES = load_data("../local/genres.pkl")
train = load_split("../local/train.pkl")
val = load_split("../local/val.pkl")
test = load_split("../local/test.pkl")
embedding = torch.load('../local/embedding.pth')
OVERVIEWS_ENCODED = load_data("../local/overviews_encoded.pkl")
TITLES_ENCODED = load_data("../local/titles_encoded.pkl")

In [ ]:
_model = Model2(embedding, hidden_dim=512, num_layers=3, cuda=True)
tomodel = TextOnlyModel(_model.encoder, _model.classifier, OVERVIEWS_ENCODED, GENRES)
loss = torch.nn.BCEWithLogitsLoss().cuda()

In [ ]:
adam = torch.optim.Adam(filter(lambda p:p.requires_grad, _model.parameters()))

sgd = torch.optim.SGD(params=filter(lambda p:p.requires_grad, tomodel.parameters()),
                            lr=1e-3, momentum=0.9)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(sgd, 'max')

optimizer = adam
scheduler = None

In [ ]:
loss_hist = []
for i in range(10):
    train_epoches(n_epochs=20, model=tomodel, train=train, loss=loss, val=val,
                  batch_size=32, optimizer=optimizer, scheduler=scheduler)
    loss_hist.append(epoch_losses)
    bn = (i+1)*50
    torch.save(tomodel.encoder, "saved/encoder_{}.pth".format(bn))
    torch.save(tomodel.classifier, "saved/rnn_cls_{}.pth".format(bn))